In [25]:
!pip uninstall scikit-learn scikit-survival
#!pip show scikit-learn
!pip install scikit-learn scikit-survival lifelines
#!pip install scikit-learn==1.1.0
#!pip install scikit-survival==0.18.0
#! pip install eli5==0.13.0

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_learn-1.5.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/sklearn/*
Proceed (Y/n)? Y
  Successfully uninstalled scikit-learn-1.5.2
Found existing installation: scikit-survival 0.23.1
Uninstalling scikit-survival-0.23.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_survival-0.23.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/sksurv/*
Proceed (Y/n)? Y
  Successfully uninstalled scikit-survival-0.23.1
Y
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scikit_survival-0.23.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.

In [1]:
### import necessary packages

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from itertools import product
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Lire le fichier en sautant la première ligne
df = pd.read_excel("/content/drive/MyDrive/GastricCancerData.xlsx", skiprows=1)
# Afficher les premières lignes du dataframe
df.head()

,AGE,SEXE,Cardiopathie,Ulceregastrique,Douleurepigastrique,Ulcero-bourgeonnant,Constipation,Denitrution,Tabac,Mucineux,Tubuleux,Infiltrant,Stenosant,Metastases,Adenopathie,Traitement,Tempsdesuivi (Mois),Deces
0,65,F,NON,NON,OUI,NON,OUI,OUI,OUI,OUI,NON,OUI,OUI,OUI,OUI,Chirurgie_Exclusive,1,OUI
1,34,F,NON,NON,OUI,NON,OUI,NON,OUI,OUI,OUI,OUI,OUI,OUI,OUI,Chirurgie_Chimiotherapie,1,OUI
2,55,H,NON,NON,OUI,NON,OUI,NON,OUI,OUI,NON,OUI,NON,OUI,OUI,Chirurgie_Exclusive,1,OUI
3,60,F,OUI,OUI,OUI,NON,OUI,NON,OUI,OUI,OUI,OUI,NON,OUI,NON,Chirurgie_Exclusive,2,NON
4,65,H,OUI,NON,OUI,NON,OUI,NON,OUI,NON,NON,OUI,OUI,NON,OUI,Chirurgie_Chimiotherapie,2,OUI


In [29]:
#df = pd.read_excel("D:/UADB/MEMOIRE/Data/GastricCancerData.xlsx")

In [30]:
#affichage des dimensions
df.shape

(337, 18)

In [31]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Identifier les colonnes catégoriques
CatCols = df.select_dtypes(include=['object']).columns
# Encodage des variables catégoriques
#Label Encoding
label_encoder = LabelEncoder()
for col in CatCols:
    df[col] = label_encoder.fit_transform(df[col].astype(str))

df.head()

,AGE,SEXE,Cardiopathie,Ulceregastrique,Douleurepigastrique,Ulcero-bourgeonnant,Constipation,Denitrution,Tabac,Mucineux,Tubuleux,Infiltrant,Stenosant,Metastases,Adenopathie,Traitement,Tempsdesuivi (Mois),Deces
0,65,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1
1,34,0,0,0,1,0,1,0,1,1,1,1,1,1,1,0,1,1
2,55,1,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,1
3,60,0,1,1,1,0,1,0,1,1,1,1,0,1,0,1,2,0
4,65,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,2,1


In [32]:
# Effacer les variables non pertinantes
clean=['SEXE','Tubuleux','Traitement','Constipation']
df=df.drop(columns=clean,axis=1)


## 2. **Gradient Boosted Survival Trees (GBST)**


In [ ]:
#!pip install --upgrade scikit-survival

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

**Données employées**  
Les variables cliniques et tumorales retenues pour l'analyse du risque de décès chez les patients atteints d'un cancer de l'estomac ont été sélectionnées selon leur pertinence statistique dans le modèle de Cox et leur contribution explicative mesurée par les valeurs SHAP. Parmi ces variables figuraient :**l'âge des patients**, la présence de **comorbidités (cardiopathies)**, des **symptômes cliniques** (douleur épigastrique, ulcère gastrique, lésion ulcéro-bourgeonnante), des **facteurs liés au mode de vie** (tabagisme, dénutrition), ainsi que des **caractéristiques tumorales** spécifiques (forme mucineuse, infiltration, sténose, métastases et adénopathies). Ces critères ont été retenus car ils ont systématiquement démontré, à travers le modèle de Cox et leurs valeurs SHAP positives, une influence significative sur la réduction de la survie. La robustesse de ces variables, validée à la fois par une approche statistique classique et par une analyse d'interprétabilité moderne (SHAP), fournit une base solide pour les explorations ultérieures.
Dans ce chapitres, nous étendrons cette analyse à des approches de *machine learning*, afin de comparer les performances prédictives et les insights cliniques entre méthodes traditionnelles et algorithmes avancés.  

In [34]:
n_censored = df.shape[0] - df["Deces"].sum()
print(f"{n_censored / df.shape[0] * 100:.1f}% des enregistrements sont censurés")

33.8% des enregistrements sont censurés


## **Optimisation des hyperparamètres**  
Pour maximiser la performance prédictive des modèles *Random Survival Forest* et *Gradient Boosting Survival*, une optimisation systématique des hyperparamètres a été réalisée à l'aide d'une **recherche exhaustive par grille (Grid Search)** couplée à une **validation croisée sur 5 plis**. Les hyperparamètres testés incluaient des éléments clés comme le nombre d'arbres (*n_estimators* : 50, 100, 200), la profondeur maximale des arbres (*max_depth* : 3, 5, 10), et des critères de division (*min_samples_split*, *min_samples_leaf*). Pour le *Random Survival Forest*, l'impact du nombre maximal de fonctionnalités (*max_features*) a également été exploré, tandis que le *Gradient Boosting* a intégré une analyse du taux d'apprentissage (*learning_rate* : 0.01, 0.1, 0.2). La sélection finale des combinaisons optimales s'est basée sur l'**indice de concordance** (*C-index*), une métrique centrale en analyse de survie pour évaluer la cohérence entre les risques prédits et les temps observés. `Cette approche rigoureuse a permis d'identifier les configurations les plus robustes pour chaque modèle, garantissant à la fois la stabilité des prédictions et la prévention du surajustement. Les résultats détaillés de cette optimisation serviront de base pour la comparaison des performances avec les modèles statistiques classiques dans les sections suivantes.`

**Données employées**  
Pour optimiser les performances des modèles *Random Survival Forest* et *Gradient Boosting Survival Tree*, une recherche systématique des hyperparamètres a été réalisée via une approche de *Grid Search* avec validation croisée (*GridSearchCV*). Pour chaque modèle, une grille d'hyperparamètres clés a été définie :  
- **Random Survival Forest** : nombre d'arbres (`n_estimators` : 50, 100, 200), profondeur maximale (`max_depth` : 3, 5, 10), critères de division des nœuds (`min_samples_split`, `min_samples_leaf`) et nombre de variables par split (`max_features`).  
- **Gradient Boosting** : taux d'apprentissage (`learning_rate` : 0.01, 0.1, 0.2), nombre d'itérations (`n_estimators`), et paramètres de complexité de l’arbre (`max_depth`, `min_samples_split`, `min_samples_leaf`).  

La sélection a été guidée par l’indice de concordance (*C-index*), mesurant la capacité du modèle à prédire correctement l’ordre des événements (décès). Une validation croisée en *5 folds* a été appliquée pour chaque combinaison d’hyperparamètres, garantissant une évaluation robuste et évitant le surajustement. Les configurations optimales retenues ont été celles maximisant le *C-index* sur les données d’entraînement. Cette méthodologie rigoureuse a permis de comparer équitablement les deux approches et de garantir leur fiabilité pour les analyses de survie ultérieures.  

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
tab_params = {
    'random_survival_forest': {
        'model': RandomSurvivalForest(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 5],
            'max_features': ["sqrt", 0.5, 1.0]  # Utilisez 'sqrt' pour les forêts aléatoires
        }
    },
    'gradient_boosting_survival': {
        'model': GradientBoostingSurvivalAnalysis(random_state=42),
        'params': {
            'learning_rate': [0.01, 0.1, 0.2],
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 5]
        }
    }
}

In [ ]:
from sklearn.model_selection import ParameterGrid
from sksurv.metrics import concordance_index_censored

def custom_grid_search_cv(model, param_grid, X_train, y_train, cv=5):
    best_score = -np.inf
    best_params = None
    for params in ParameterGrid(param_grid):
        model.set_params(**params)
        scores = []
        # Validation croisée manuelle
        for train_idx, val_idx in KFold(n_splits=cv).split(X_train):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]
            model.fit(X_tr, y_tr)
            ci = concordance_index_censored(y_val["event"], y_val["time"], model.predict(X_val))[0]
            scores.append(ci)
        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = params
    return best_params, best_score

In [ ]:
from sklearn.model_selection import KFold

results = []
for model_name, mp in tab_params.items():
    best_params, best_score = custom_grid_search_cv(mp['model'], mp['params'], X_train, y_train)
    results.append({
        'model': model_name,
        'best_score': best_score,
        'best_params': best_params
    })

results_df = pd.DataFrame(results)
results_df

Les résultats obtenus avec la méthode **RandomizedSearchCV** ont permis de trouver les meilleurs paramètres pour deux modèles de prédiction de survie : **Random Survival Forest** et **Gradient Boosting Survival Analysis**.  


Pour le **Random Survival Forest**, les paramètres sélectionnés indiquent que l'algorithme a utilisé **50 arbres**, chacun ayant une profondeur maximale de **10 niveaux**. À chaque division d’un arbre, **seulement 50 % des variables** ont été prises en compte, ce qui a aidé à éviter le surapprentissage. De plus, un minimum de **10 observations** était nécessaire pour qu'un nœud se divise, et chaque feuille contenait au moins **une observation**.  

Concernant le **Gradient Boosting Survival Analysis**, l’algorithme a utilisé **100 arbres** avec une profondeur maximale de **5 niveaux**. Le taux d'apprentissage a été fixé à **0.1**, ce qui signifie que le modèle s'est ajusté progressivement pour éviter d’apprendre trop vite et risquer de perdre en précision. Comme pour le premier modèle, il fallait **au moins une observation par feuille** et **10 observations pour qu’un nœud se divise**.  

Ces réglages ont permis d’optimiser la performance des modèles pour mieux prédire le temps de survie des individus en fonction des données disponibles.

#### **Reoptimisation des hyperparametres**

In [44]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import KFold
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
tab_params = {
    'random_survival_forest': {
        'model': RandomSurvivalForest(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 5],
            'max_features': ["sqrt", 0.5, 1.0]  # Utilisez 'sqrt' pour les forêts aléatoires
        }
    },
    'gradient_boosting_survival': {
        'model': GradientBoostingSurvivalAnalysis(random_state=42),
        'params': {
            'learning_rate': [0.01, 0.1, 0.2],
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 5]
        }
    }
}

In [45]:
from sklearn.model_selection import ParameterGrid
from sksurv.metrics import concordance_index_censored

def custom_grid_search_cv(model, param_grid, X_train, y_train, cv=5):
    best_score = -np.inf
    best_params = None
    for params in ParameterGrid(param_grid):
        model.set_params(**params)
        scores = []
        # Validation croisée manuelle
        for train_idx, val_idx in KFold(n_splits=cv).split(X_train):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]
            model.fit(X_tr, y_tr)
            ci = concordance_index_censored(y_val["event"], y_val["time"], model.predict(X_val))[0]
            scores.append(ci)
        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = params
    return best_params, best_score

In [47]:
from sklearn.model_selection import KFold

results = []
for model_name, mp in tab_params.items():
    best_params, best_score = custom_grid_search_cv(mp['model'], mp['params'], X_train, y_train)
    results.append({
        'model': model_name,
        'best_score': best_score,
        'best_params': best_params
    })

results_df = pd.DataFrame(results)
results_df

,model,best_score,best_params
0,random_survival_forest,0.844741,"{'max_depth': 10, 'max_features': 0.5, 'min_sa..."
1,gradient_boosting_survival,0.850937,"{'learning_rate': 0.1, 'max_depth': 5, 'min_sa..."


## **II. Gradient Boosting Survival Trees**

In [52]:
# Lire le fichier en sautant la première ligne
df = pd.read_excel("/content/drive/MyDrive/GastricCancerData.xlsx", skiprows=1)
# Afficher les premières lignes du dataframe
df.head()
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Identifier les colonnes catégoriques
CatCols = df.select_dtypes(include=['object']).columns
# Encodage des variables catégoriques
#Label Encoding
label_encoder = LabelEncoder()
for col in CatCols:
    df[col] = label_encoder.fit_transform(df[col].astype(str))

In [53]:
# Effacer les variables non pertinantes
clean=['SEXE','Tubuleux','Traitement','Constipation']
df=df.drop(columns=clean,axis=1)

In [56]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df[["AGE", 'Cardiopathie', 'Ulceregastrique', 'Douleurepigastrique',
       'Ulcero-bourgeonnant', 'Denitrution', 'Tabac',
       'Mucineux', 'Infiltrant', 'Stenosant', 'Metastases',
       'Adenopathie']].values

T = df['Tempsdesuivi (Mois)'].values
E = df['Deces'].values

# Préparer les données pour le modèle de survie
# Créer une structure pour le statut de censure (event/censure)
y = np.array([(bool(event), time) for event, time in zip(E, T)],
             dtype=[('event', 'bool'), ('time', 'float')])

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sksurv.ensemble import GradientBoostingSurvivalAnalysis

# Créer et entraîner le modèle avec les paramètres par défaut
gbst = GradientBoostingSurvivalAnalysis(random_state=42)

# Entraînement du modèle
gbst.fit(X_train, y_train)

GradientBoostingSurvivalAnalysis(random_state=42)

In [57]:
#Évaluer le modèle
train_score = gbst.score(X_train, y_train)
test_score = gbst.score(X_test, y_test)

print(f"Score de concordance (train) : {train_score:.2f}")
print(f"Score de concordance (test) : {test_score:.2f}")

Score de concordance (train) : 0.91
Score de concordance (test) : 0.86


In [62]:
from sklearn.inspection import permutation_importance

# Appliquer l'importance par permutation
result = permutation_importance(
    gbst, X_test, y_test,
    n_repeats=10, random_state=42
)

# Créer un DataFrame pour organiser les résultats
importance_df = pd.DataFrame({
    "Feature": X_test.columns,
    "Importance": result.importances_mean,
    "Std Dev": result.importances_std
}).sort_values(by="Importance", ascending=False)

print(importance_df)

# Préparer les couleurs : rouge pour les valeurs négatives, bleu pour les positives
colors = ["blue" if val > 0 else "red" for val in importance_df["Importance"]]

# Visualiser les résultats
plt.figure(figsize=(10, 6))
bars = plt.barh(
    importance_df["Feature"],
    importance_df["Importance"],
    color=colors,
    edgecolor="black"
)
plt.gca().invert_yaxis()  # Inverser l'axe pour que la plus importante soit en haut
plt.xlabel("Importance moyenne (Permutation)")
plt.title("Importance des variables par permutation")

# Ajouter les étiquettes sur les pics des barres
for bar in bars:
    width = bar.get_width()  # Longueur de la barre
    label_x_pos = width + 0.0001 if width > 0 else width - 0.0001  # Position de l'étiquette
    plt.text(
        label_x_pos, bar.get_y() + bar.get_height() / 2,
        f"{width:.3f}",  # Afficher la valeur avec deux décimales
        va="center", ha="left" if width > 0 else "right",
        color="black", fontsize=10
    )

plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [51]:
results_df.loc[1,"best_params"]

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 100}

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df[["AGE",'SEXE', 'Cardiopathie', 'Ulceregastrique', 'Douleurepigastrique',
       'Ulcero-bourgeonnant', 'Constipation', 'Denitrution', 'Tabac',
       'Mucineux', 'Tubuleux', 'Infiltrant', 'Stenosant', 'Metastases',
       'Adenopathie', 'Traitement'
]].values
T = df['Tempsdesuivi (Mois)'].values
E = df['Deces'].values

# Préparer les données pour le modèle de survie
# Créer une structure pour le statut de censure (event/censure)
y = np.array([(bool(event), time) for event, time in zip(E, T)],
             dtype=[('event', 'bool'), ('time', 'float')])

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sksurv.ensemble import GradientBoostingSurvivalAnalysis

# Créer et entraîner le modèle avec les paramètres par défaut
gbst = GradientBoostingSurvivalAnalysis(learning_rate=0.2,
                                              max_depth=5,
                                              min_samples_leaf=1,
                                              min_samples_split=10,
                                              n_estimators=100)

# Entraînement du modèle
gbst.fit(X_train, y_train)

#Évaluer le modèle
train_score = gbst.score(X_train, y_train)
test_score = gbst.score(X_test, y_test)

print(f"Score de concordance (train) : {train_score:.2f}")
print(f"Score de concordance (test) : {test_score:.2f}")

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df[["AGE","Tabac", "Metastases", "Infiltrant", "Adenopathie", "Ulcero-bourgeonnant",
        "Ulceregastrique", "Stenosant", "Denitrution"]].values

T = df['Tempsdesuivi (Mois)'].values
E = df['Deces'].values

# Préparer les données pour le modèle de survie
# Créer une structure pour le statut de censure (event/censure)
y = np.array([(bool(event), time) for event, time in zip(E, T)],
             dtype=[('event', 'bool'), ('time', 'float')])

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sksurv.ensemble import GradientBoostingSurvivalAnalysis

# Créer et entraîner le modèle avec les paramètres par défaut
gbst = GradientBoostingSurvivalAnalysis(learning_rate=0.2,
                                              max_depth=3,
                                              min_samples_leaf=2,
                                              min_samples_split=2,
                                              n_estimators=100)

# Entraînement du modèle
gbst.fit(X_train, y_train)

#Évaluer le modèle
train_score = gbst.score(X_train, y_train)
test_score = gbst.score(X_test, y_test)

print(f"Score de concordance (train) : {train_score:.2f}")
print(f"Score de concordance (test) : {test_score:.2f}")

In [ ]:
# Calcul des C-index
c_index_train = gbst.score(X_train, y_train)
c_index_test = gbst.score(X_test, y_test)

print(f'Training data C-index: {c_index_train:.2f}')
print(f'Testing data C-index: {c_index_test:.2f}')

# Bootstrap pour intervalle de confiance du C-index
n_bootstraps = 1000
random_state = 42
bootstrap_c_indexes = []

np.random.seed(random_state)
for _ in range(n_bootstraps):
    X_test_sample, y_test_sample = resample(X_test, y_test)
    c_index = gbst.score(X_test_sample, y_test_sample)
    bootstrap_c_indexes.append(c_index)

# Calcul de l'intervalle de confiance
alpha = 0.95
lower_bound = np.percentile(bootstrap_c_indexes, (1 - alpha) / 2 * 100)
upper_bound = np.percentile(bootstrap_c_indexes, (1 + alpha) / 2 * 100)
mean_c_index = np.mean(bootstrap_c_indexes)

print(f"Bootstrap C-index intervalle ({alpha*100:.0f}% IC): [{lower_bound:.2f}, {upper_bound:.2f}]")
print(f"Bootstrap C-index moyenne: {mean_c_index:.2f}")


Performance du modèle :

C-index en entraînement : 0.90  Très haute performance sur les données d'entraînement.

C-index en test : 0.86  Bonne généralisation à de nouvelles données.

Robustesse du modèle :

Intervalle de confiance bootstrap (95% IC) : [0.81, 0.90]  La performance du modèle est stable et fiable, avec une performance minimale garantie de 0.81.

Moyenne bootstrap C-index : 0.86  Confirme la performance observée en test.

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored, integrated_brier_score
from sklearn.model_selection import train_test_split

# Calcul des métriques
# Concordance Index
c_index = concordance_index_censored(
    y_test["event"], y_test["time"], gbst.predict(X_test)
)[0]

# Brier Score intégré
# Limiter les points temporels à l'intervalle de suivi des données de test (exclure les bords)
min_time = y_test["time"].min() + 1e-8  # Légèrement au-dessus du min
max_time = y_test["time"].max() - 1e-8  # Légèrement en-dessous du max
time_points = np.linspace(min_time, max_time, 100)

# Extraction des prédictions de survie
survival_predictions = gbst.predict_survival_function(X_test)

# Convertir les prédictions en tableau 2D (probabilités à chaque point de temps)
survival_probabilities = np.asarray(
    [[fn(t) for t in time_points] for fn in survival_predictions]
)

# Calcul du Brier score intégré
brier_score_value = integrated_brier_score(y_train, y_test, survival_probabilities, time_points)

# Création d'un tableau des métriques avec 2 décimales
metrics_table = pd.DataFrame({
    "Metric": ["Brier Score intégré", "C-Index"],
    "Value": [f"{brier_score_value:.2f}", f"{mean_c_index:.2f}"]
})

print(metrics_table)